# Recuperação de Portadora

Este notebook contém algumas implementações dos principais algoritmos que realizam a recuperação de portadora em receptores ópticos digitais coerentes, com a finalidade de compensar o deslocamento de frequência da portadora e o ruído de fase.

## 6.1 Frequency Recovery

Existem dois efeitos de fase principais que afetam os sistemas ópticos coerentes digitais. O primeiro é um deslocamento de frequência da portadora, o segundo é o ruído de fase.

Assumindo uma detecção coerente intradina, os lasers transmissores e osciladores locais não operam exatamente na mesma frequência, mas exibem um deslocamento que pode atingir a ordem de alguns gigahertz. Após o front-end óptico, esse deslocamento de frequência é traduzido para o domínio elétrico, levando a uma perda completa da referência de fase para o processo de decisão.

Assumindo uma equalização perfeita e demultiplexação de polarização, um sinal equalizado $y[k]$, prejudicado por um deslocamento de frequência, ruído de fase e ruído aditivo, pode ser expresso como:

$$ y[k]=s[k]e^{j(\theta[k]+k2\pi\Delta f T_{s})}+\eta[k]$$

Onde $s[k]$ é o sinal transmitido, $\theta[k]$ é o ruído de fase, $\Delta f$ é o deslocamento de frequência da portadora, $T_s$ é o período do símbolo e $\eta [k]$ é o ruído gaussiano branco aditivo (AWGN). Como o ruído de fase e os deslocamentos de frequência da portadora têm propriedades diferentes, eles são compensados ​​por algoritmos diferentes. Os algoritmos de recuperação de frequência estimam e compensam $\Delta f$ , enquanto os algoritmos de recuperação de fase estimam e compensam $\theta[k]$.